### Import packages

In [1]:
# Import packages
import os
import rasterio
import fiona
from rasterio.mask import mask

### Open the images

In [3]:
# Define the directory containing the updated image
images_dir = r'D:\uMzi scene\data\EMIT\EMIT_Scenes'

# Loop through each image in the directory
for image_file in os.listdir(images_dir):
    if image_file.endswith('.TIF') or image_file.endswith('.tif'):  # Check for both upper and lower case extensions
       
        # Open the image
        with rasterio.open(os.path.join(images_dir, image_file)) as img:
            # Print the number of bands
            print(f"Image '{image_file}' has {img.count} bands.")

Image 'EMIT_L2A_001_20240308T070913_2406805_011.tif' has 285 bands.
Image 'EMIT_L2A_20240229T101956_2406007_017.tif' has 285 bands.
Image 'EMIT_L2A_TIF_001_20240229T101944_2406007_016.tif' has 285 bands.


### Clip images to ROI

In [4]:
# Define the directory containing the updated images and the ROI shapefile
images_dir = r'D:\uMzi scene\data\EMIT\EMIT_Scenes'
umzim_roi = r'C:\Users\SkosanaT\OneDrive - Stellenbosch University\MAPWAPS\DataChapter1\uMzi_ROI\roi\ROI.shp'

# Create an output folder inside the images directory
output_dir = os.path.join(images_dir, 'clipped')
# Check if the output directory exists, if not, create it
os.makedirs(output_dir, exist_ok=True)

# Loop through each image in the images directory
for image_file in os.listdir(images_dir):
    if image_file.endswith('.TIF') or image_file.endswith('.tif'):  # Check for both upper and lower case extensions
        
        # Open the image
        with rasterio.open(os.path.join(images_dir, image_file)) as img:
            # Print the number of bands
            print(f"Image '{image_file}' has {img.count} bands.")
            
            # Read the ROI shapefile
            with fiona.open(umzim_roi, 'r') as shapefile:
                shapes = [feature['geometry'] for feature in shapefile]
                
                # Clip the image using the ROI shapefile
                out_image, out_transform = mask(img, shapes, crop=True)
                out_meta = img.meta.copy()
                
                # Update metadata after clipping
                out_meta.update({
                    "driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform
                })
                
                # Define the output file path
                output_file = os.path.join(output_dir, f"clipped_{image_file}")
                
                # Save the clipped image
                with rasterio.open(output_file, "w", **out_meta) as dest:
                    dest.write(out_image)
                    
            print(f"Image '{image_file}' clipped and saved to '{output_file}'.")


Image 'EMIT_L2A_001_20240308T070913_2406805_011.tif' has 285 bands.
Image 'EMIT_L2A_001_20240308T070913_2406805_011.tif' clipped and saved to 'D:\uMzi scene\data\EMIT\EMIT_Scenes\clipped\clipped_EMIT_L2A_001_20240308T070913_2406805_011.tif'.
Image 'EMIT_L2A_20240229T101956_2406007_017.tif' has 285 bands.
Image 'EMIT_L2A_20240229T101956_2406007_017.tif' clipped and saved to 'D:\uMzi scene\data\EMIT\EMIT_Scenes\clipped\clipped_EMIT_L2A_20240229T101956_2406007_017.tif'.
Image 'EMIT_L2A_TIF_001_20240229T101944_2406007_016.tif' has 285 bands.
Image 'EMIT_L2A_TIF_001_20240229T101944_2406007_016.tif' clipped and saved to 'D:\uMzi scene\data\EMIT\EMIT_Scenes\clipped\clipped_EMIT_L2A_TIF_001_20240229T101944_2406007_016.tif'.
